In [7]:
import pandas as pd
import os
import glob
import numpy as np

In [8]:
fileloc = r"G:\My Drive\Tom-Tanvi\Shared With Bing,Tom and Tanvi\Video Analysis\DeepLabCut-ImageAnalysis\take5\outputFromDLC\VideoResults\EntireDataSet"

annotation_folder = r"G:\My Drive\Tom-Tanvi\Shared With Bing,Tom and Tanvi\Moth Annotations\CSV for Video Annotations"


In [9]:
filelist = glob.glob(fileloc + r"\*.h5")
# filelist

In [10]:
annotation_filelist = glob.glob(annotation_folder + r"\*MothInOut.csv")
# annotation_filelist

In [35]:
# run in a loop and pull up the data from all 8 files together - make a uber DF

uber_df = pd.DataFrame()

for f in annotation_filelist:
    annotation = pd.read_csv(f)
    
    filename = os.path.basename(f)
    file_matched = [file for file in filelist if filename[:-14] in file]
    
    temp = pd.read_hdf(file_matched[0])
    DLC = temp.loc[slice(None),(slice(None),'proboscisTip')].copy()
    
    DLC.columns = DLC.columns.droplevel(level = [0,1])
    
    # create new pandas series that will have all the annotation for each frame

    label = pd.Series(index = DLC.index, name = 'label')

    # run through annotation file each line -
    for line in annotation.index:
        label.loc[annotation.loc[line, 'MothIN']: annotation.loc[line, 'MothOut']] = annotation.loc[line, 'Annotations']

    label[label.isnull()] = 'NA'
    DLC.loc[:,'label']=label

    # convert all points within NaN zone (5,5) to NaN
    DLC.loc[DLC['x']<5, 'x']=np.nan
    DLC.loc[DLC['y']<5, 'y']=np.nan
    
    # add mothID
    moth = pd.Series(index = DLC.index, name = 'mothID')
    moth = filename[:-14]
    DLC.loc[:,'mothID']=moth
    
    # add frame number as a separate column
    DLC.loc[:,'frameNum'] = DLC.index.values
    
    uber_df = uber_df.append(DLC, ignore_index=True)
    print(moth + '_' + str(len(DLC.index)))
    print(len(uber_df.index))
    
#     del DLC
    del temp

c-2_m4_190757
190757
c-3_m11_195651
386408
c-10_m13_110013
496421
c-1_m4_166720
663141
c-2_m11_186751
849892
c-3_m3_178001
1027893
c-10_m15_191970
1219863
c-1_m6_172935
1392798


In [36]:
uber_df.head()

coords,x,y,likelihood,label,mothID,frameNum
0,NaN,NaN,0.000002,NA,c-2_m4,0
1,322.195326,306.575773,0.000005,NA,c-2_m4,1
2,NaN,NaN,0.000002,NA,c-2_m4,2
3,NaN,NaN,0.000002,NA,c-2_m4,3
4,NaN,NaN,0.000002,NA,c-2_m4,4


In [37]:
uber_df.to_csv('../dataFolders/DLCtrackingResults/allDLCtrackingData.csv')